In [1]:
import os
import sys
import matplotlib.pyplot as plt

from datetime import datetime
import random, operator, csv
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sn
#import ray#, psutil
from multiprocessing import Pool

import tellurium as te

In [2]:
with open("egfrActivation.sb") as f:
    egfrActivationModel = f.read()

model = te.loada(egfrActivationModel)
species = model.getFloatingSpeciesIds()
#model.reset()

In [3]:
egf = np.array([0.2, 0.4, 1.0, 2.5, 5.0, 10.0, 20.0, 100.0])
pegfr = np.array([[1.538, 1.639, 2.355, 1.899, 1.935, 1.790, 1.700, 0.781, 3.072],
[2.244, 1.943, 1.395, 1.749, 2.028, 2.039, 2.087, 1.108, 2.858],
[1.501, 2.935, 4.037, 5.768, 5.986, 7.527, 6.889, 7.812, 7.490],
[2.899, 9.543, 13.878, 18.940, 23.153, 27.694, 28.414, 34.901, 37.434],
[2.504, 13.263, 21.242, 31.053, 34.163, 37.691, 36.178, 47.017, 46.832],
[2.642, 12.110, 15.855, 28.133, 30.337, 29.640, 32.854, 31.818, 44.066],
[2.069, 18.130, 13.089, 24.388, 29.611, 36.226, 32.383, 37.782, 38.338],
[1.568, 20.861, 38.950, 38.886, 32.507, 32.372, 31.802, 30.657, 38.321]])

In [4]:
t0 = 0
tf = 80
tnum = 9
egfLevels = len(egf)

def getResidual(pars, model = model):
    model = setParameter(model, pars)
    residual = 1
    for i in range(egfLevels):
        model.reset()
        model.E = egf[i]
        eq = model.simulate(t0, tf, tnum)
        tempSc = np.sqrt(np.sum(((eq["[Rp]"] - pegfr[i])/(0.1 * pegfr[i]))**2) / 9)
        residual *= tempSc
    residual **= 1/egfLevels
    
    return residual

#def getObs(model, pars):
    

def setParameter(model, pars):
    model.k1 = pars[0]
    model.k_1 = pars[1]
    model.kt = pars[2]
    model.ke = pars[3]
    model.Vr = pars[4]
    model.kx = pars[5]
    model.kh = pars[6]
    model.k2 = pars[7]
    model.k_2 = pars[8]
    model.k3 = pars[9]
    model.k_3 = pars[10]
    return model

In [5]:
def getTempLin(lin,prob,scaler):
    dims = lin.shape
    tempLin = []
    for i in range(dims[0]):
        tempi = []
        randi = random.randrange(dims[1])
        choices = random.sample(range(dims[0]),3)
        while i in choices:
            choices = random.sample(range(dims[0]),3)
        for j in range(dims[1]):
            if random.random() < prob or j == randi :
                newPar = lin[choices[0],j] + scaler * (lin[choices[1],j] - lin[choices[2],j])
                if newPar < -4 or newPar > 2:
                    newPar = random.uniform(-8, 8)
                tempi.append(newPar)
            else:
                tempi.append(lin[i,j])
        tempLin.append(tempi)
    return(np.array(tempLin))

In [6]:
#pool = Pool(processes=4)

In [7]:
#pool.close()

In [8]:
gen = 0
batchsize = 12
genNum = 1000000
epoch = 10000
interval = 1000
scaling = 0.9
error = 1.0e32
threshold = 0.1
trackfile = "tracking.csv"
moniterfile = "monitor.csv"

In [9]:
lin3 = []
lineages = []
for i in range(batchsize):
    iLin3 = np.random.uniform(-4, 2, 3)
    lin3.append(iLin3)
    lineages.append(np.array([0.0016, iLin3[0], 0.0012, 0.0033, 
                         0.242, 1, 0.0004, 0.082, iLin3[1], 1.2, iLin3[2]]))
lin3 = np.array(lin3)
lineages = np.array(lineages)

In [10]:
scores = [getResidual(10 ** iLin) for iLin in lineages]
scores

[9.842127021836136,
 9.783177549313955,
 9.984518642525922,
 9.894190088521876,
 9.80937022804237,
 9.996802521374734,
 9.967965741888621,
 9.995361088057855,
 9.777817798110846,
 9.786033717986182,
 9.895481882743418,
 9.780388542438464]

In [ ]:
count = 0
while gen <= genNum or error > threshold:
    crossProb = 0.25 * ( 2 - random.random() )
    tempLin3 = getTempLin(lin3,crossProb,scaling)
    tempLin = np.array([[0.0016, iLin3[0], 0.0012, 0.0033, 
                         0.242, 1, 0.0004, 0.082, iLin3[1], 
                         1.2, iLin3[2]] for iLin3 in tempLin3])
    tempScores = [getResidual(10 ** iLin) for iLin in tempLin]

    for i in range(batchsize):
        if tempScores[i] < scores[i]:
            scores[i] = tempScores[i]
            lineages[i] = tempLin[i]
            lin3[i] = tempLin3[i]
            count += 1
    bestindex, bestfit = min(enumerate(scores), key=operator.itemgetter(1))

    if bestfit < error:
        error = bestfit
        with open(trackfile, 'a') as output:
            writer = csv.writer(output)
            writer.writerow([gen, bestfit] + lin3[bestindex].tolist())
    if gen % interval == 0:
        with open(moniterfile, 'a') as output:
            writer = csv.writer(output)
            writer.writerow([str(datetime.now()), gen, bestfit, np.mean(scores), count, np.amin(tempScores), np.mean(tempScores)])
        count = 0

    gen += 1